## Preparation

### Installation

In [1]:
# Building sources
!git clone git@github.com:oaqa/FlexNeuART.git

Cloning into 'FlexNeuART'...
remote: Enumerating objects: 289, done.
remote: Counting objects: 100% (289/289), done.
remote: Compressing objects: 100% (183/183), done.
remote: Total 14643 (delta 129), reused 193 (delta 66), pack-reused 14354
Receiving objects: 100% (14643/14643), 31.47 MiB | 31.97 MiB/s, done.
Resolving deltas: 100% (8994/8994), done.


In [2]:
cd FlexNeuART

/Users/leo/Downloads/test/FlexNeuART


In [3]:
!./install_packages.sh

Using collection root: collections
Python packages to install:
lxml pytools torch 1.4 torchtext 0.6.0 numpy bs4 thrift 0.13.0 spacy 2.2.3 pyjnius
Installing package lxml -> 
Installing package pytools -> 
Installing package torch -> 1.4
Installing package torchtext -> 0.6.0
Installing package numpy -> 
Installing package bs4 -> 
Installing package thrift -> 0.13.0
Installing package spacy -> 2.2.3
Installing package pyjnius -> 
     |████████████████████████████████| 284 kB 6.6 MB/s eta 0:00:01


✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
Cloning into 'pytorch-pretrained-BERT-mod'...
remote: Enumerating objects: 85, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 85 (delta 27), reused 79 (delta 21), pack-reused 0
Unpacking objects: 100% (85/85), 293.34 KiB | 3.22 MiB/s, done.
running install
running bdist_egg
running egg_info
creating pytorch_pretrained_bert.egg-info
writing pytorch_pretrained_bert.egg-info/PKG-INFO
writing dependency_links to pytorch_pretrained_bert.egg-info/dependency_links.txt
writing entry points to pytorch_pretrained_bert.egg-info/entry_points.txt
writing requirements to pytorch_pretrained_bert.egg-info/requires.txt
writing top-level names to pytorch_pretrained_bert.egg-info/top_level.txt
writing manifest file 'pytorch_pretrained_bert.egg-info/SOURCES.txt'
reading manifest file 'pytorch_pretrained_bert.egg-info/SOURCES.txt'
reading man

byte-compiling build/bdist.macosx-10.9-x86_64/egg/pytorch_pretrained_bert/convert_gpt2_checkpoint_to_pytorch.py to convert_gpt2_checkpoint_to_pytorch.cpython-38.pyc
byte-compiling build/bdist.macosx-10.9-x86_64/egg/pytorch_pretrained_bert/file_utils.py to file_utils.cpython-38.pyc
byte-compiling build/bdist.macosx-10.9-x86_64/egg/pytorch_pretrained_bert/tokenization_transfo_xl.py to tokenization_transfo_xl.cpython-38.pyc
byte-compiling build/bdist.macosx-10.9-x86_64/egg/pytorch_pretrained_bert/modeling.py to modeling.cpython-38.pyc
byte-compiling build/bdist.macosx-10.9-x86_64/egg/pytorch_pretrained_bert/__main__.py to __main__.cpython-38.pyc
creating build/bdist.macosx-10.9-x86_64/egg/EGG-INFO
copying pytorch_pretrained_bert.egg-info/PKG-INFO -> build/bdist.macosx-10.9-x86_64/egg/EGG-INFO
copying pytorch_pretrained_bert.egg-info/SOURCES.txt -> build/bdist.macosx-10.9-x86_64/egg/EGG-INFO
copying pytorch_pretrained_bert.egg-info/dependency_links.txt -> build/bdist.macosx-10.9-x86_64/egg

Scanning dependencies of target snt2coocrmp
[ 13%] Linking CXX executable ../bin/symal
[ 14%] Building CXX object src/CMakeFiles/snt2coocrmp.dir/snt2cooc-reduce-mem-preprocess.cpp.o
[ 14%] Built target plain2snt
[ 16%] Building CXX object src/CMakeFiles/mgiza_lib.dir/ATables.cpp.o
[ 16%] Built target symal
[ 17%] Building C object src/CMakeFiles/mgiza_lib.dir/cmd.c.o
Scanning dependencies of target snt2cooc
[ 19%] Building CXX object src/CMakeFiles/snt2cooc.dir/snt2cooc.cpp.o
[ 20%] Building CXX object src/CMakeFiles/mgiza_lib.dir/collCounts.cpp.o
[ 22%] Linking CXX executable ../bin/snt2coocrmp
[ 22%] Built target snt2coocrmp
[ 23%] Building CXX object src/CMakeFiles/mgiza_lib.dir/Dictionary.cpp.o
Scanning dependencies of target mkcls
In file included from /Users/leo/Downloads/test/FlexNeuART/mgiza/mgizapp/src/collCounts.cpp:28:
/Users/leo/Downloads/test/FlexNeuART/mgiza/mgizapp/src/D5Tables.h:191:11: warning: 
      add explicit braces to avoid dangling else [-Wdangling-else]
       

In file included from /Users/leo/Downloads/test/FlexNeuART/mgiza/mgizapp/src/model3.cpp:22:
In file included from /Users/leo/Downloads/test/FlexNeuART/mgiza/mgizapp/src/model3.h:51:
In file included from /Users/leo/Downloads/test/FlexNeuART/mgiza/mgizapp/src/transpair_modelhmm.h:35:
In file included from /Users/leo/Downloads/test/FlexNeuART/mgiza/mgizapp/src/hmm.h:53:
/Users/leo/Downloads/test/FlexNeuART/mgiza/mgizapp/src/HMMTables.h:185:15: warning: 
      add explicit braces to avoid dangling else [-Wdangling-else]
            } else {
              ^
In file included from /Users/leo/Downloads/test/FlexNeuART/mgiza/mgizapp/src/model3.cpp:22:
In file included from /Users/leo/Downloads/test/FlexNeuART/mgiza/mgizapp/src/model3.h:55:
/Users/leo/Downloads/test/FlexNeuART/mgiza/mgizapp/src/D5Tables.h:191:11: warning: 
      add explicit braces to avoid dangling else [-Wdangling-else]
          else
          ^
/Users/leo/Downloads/test/FlexNeuART/mgiza/mgizapp/src/model3.cpp:735:9: warning

...

[INFO] 
[INFO] --- maven-surefire-plugin:2.12.4:test (default-test) @ RankLib ---
[INFO] Surefire report directory: /Users/leo/Downloads/test/FlexNeuART/lemur-code-r2792-RankLib-trunk/target/surefire-reports

-------------------------------------------------------
 T E S T S
-------------------------------------------------------
Running ciir.umass.edu.SimpleTest
Tests run: 1, Failures: 0, Errors: 0, Skipped: 0, Time elapsed: 0.073 sec
Running ciir.umass.edu.eval.EvaluatorTest

Discard orig. features
Training data:	/var/folders/zx/6p0tyww13rn7xbr8p28vpdbm0000gn/T/ranklib15001978068626546613tmp
Feature vector representation: Dense.
Ranking method:	Coordinate Ascent
Feature description file:	Unspecified. All features will be used.
Train metric:	map
Test metric:	map
Feature normalization: No
Model file: /var/folders/zx/6p0tyww13rn7xbr8p28vpdbm0000gn/T/ranklib5456253495915585467tmp

[+] Coordinate Ascent's Parameters:
No. of random restarts: 5
No. of iterations to search in each direction:

293     | 1.0       | 
294     | 1.0       | 
295     | 1.0       | 
296     | 1.0       | 
297     | 1.0       | 
298     | 1.0       | 
299     | 1.0       | 
300     | 1.0       | 
301     | 1.0       | 
302     | 1.0       | 
303     | 1.0       | 
304     | 1.0       | 
305     | 1.0       | 
306     | 1.0       | 
307     | 1.0       | 
308     | 1.0       | 
309     | 1.0       | 
310     | 1.0       | 
311     | 1.0       | 
312     | 1.0       | 
313     | 1.0       | 
314     | 1.0       | 
315     | 1.0       | 
316     | 1.0       | 
317     | 1.0       | 
318     | 1.0       | 
319     | 1.0       | 
320     | 1.0       | 
321     | 1.0       | 
322     | 1.0       | 
323     | 1.0       | 
324     | 1.0       | 
325     | 1.0       | 
326     | 1.0       | 
327     | 1.0       | 
328     | 1.0       | 
329     | 1.0       | 
330     | 1.0       | 
331     | 1.0       | 
332     | 1.0       | 
333     | 1.0       | 
334     | 1.0       | 
335     | 1.0       | 
336     | 1

673     | 1.0       | 
674     | 1.0       | 
675     | 1.0       | 
676     | 1.0       | 
677     | 1.0       | 
678     | 1.0       | 
679     | 1.0       | 
680     | 1.0       | 
681     | 1.0       | 
682     | 1.0       | 
683     | 1.0       | 
684     | 1.0       | 
685     | 1.0       | 
686     | 1.0       | 
687     | 1.0       | 
688     | 1.0       | 
689     | 1.0       | 
690     | 1.0       | 
691     | 1.0       | 
692     | 1.0       | 
693     | 1.0       | 
694     | 1.0       | 
695     | 1.0       | 
696     | 1.0       | 
697     | 1.0       | 
698     | 1.0       | 
699     | 1.0       | 
700     | 1.0       | 
701     | 1.0       | 
702     | 1.0       | 
703     | 1.0       | 
704     | 1.0       | 
705     | 1.0       | 
706     | 1.0       | 
707     | 1.0       | 
708     | 1.0       | 
709     | 1.0       | 
710     | 1.0       | 
711     | 1.0       | 
712     | 1.0       | 
713     | 1.0       | 
714     | 1.0       | 
715     | 1.0       | 
716     | 1

Test Ranker: 8

Discard orig. features
Training data:	/var/folders/zx/6p0tyww13rn7xbr8p28vpdbm0000gn/T/ranklib672454452183854136tmp
Feature vector representation: Dense.
Ranking method:	Random Forests
Feature description file:	Unspecified. All features will be used.
Train metric:	map
Test metric:	map
Feature normalization: No
Model file: /var/folders/zx/6p0tyww13rn7xbr8p28vpdbm0000gn/T/ranklib9534616344457594800tmp

[+] Random Forests's Parameters:
No. of bags: 10
Sub-sampling: 1.0
Feature-sampling: 1.0
No. of trees: 1
No. of leaves: 100
No. of threshold candidates: 256
Learning rate: 0.1

Reading feature file [/var/folders/zx/6p0tyww13rn7xbr8p28vpdbm0000gn/T/ranklib672454452183854136tmp]... [Done.]            
(1 ranked lists, 200 entries read)
Initializing... [Done]
------------------------------------
Training starts...
------------------------------------
bag       | MAP-B     | MAP-OOB     | 
------------------------------------
b[1]      | 1.0       | 
b[2]      | 1.0       | 
b[

Test Ranker: 6

Discard orig. features
Training data:	/var/folders/zx/6p0tyww13rn7xbr8p28vpdbm0000gn/T/ranklib692519236583645309tmp
Feature vector representation: Dense.
Ranking method:	LambdaMART
Feature description file:	Unspecified. All features will be used.
Train metric:	map
Test metric:	map
Feature normalization: No
Model file: /var/folders/zx/6p0tyww13rn7xbr8p28vpdbm0000gn/T/ranklib5992859941490874005tmp

[+] LambdaMART's Parameters:
No. of trees: 1
No. of leaves: 100
No. of threshold candidates: 256
Min leaf support: 1
Learning rate: 0.1
Stop early: -1 rounds without performance gain on validation data

Reading feature file [/var/folders/zx/6p0tyww13rn7xbr8p28vpdbm0000gn/T/ranklib692519236583645309tmp]... [Done.]            
(1 ranked lists, 200 entries read)
Initializing... [Done]
---------------------------------
Training starts...
---------------------------------
#iter   | MAP-T     | MAP-V     | 
---------------------------------
1       | 1.0       | 
--------------------

In [4]:
!./build.sh

 BUILDING main codebase 
 log: /Users/leo/Downloads/test/FlexNeuART/main.build.log
BUILD IS COMPLETE!


### Downloading demo data

In [5]:
!wget boytsov.info/datasets/flecsneurt-demo-2020-04-07.tar.bz2 

--2021-02-04 02:12:20--  http://boytsov.info/datasets/flecsneurt-demo-2020-04-07.tar.bz2
Resolving boytsov.info (boytsov.info)... 69.60.127.165
Connecting to boytsov.info (boytsov.info)|69.60.127.165|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44021208 (42M) [application/x-bzip2]
Saving to: ‘flecsneurt-demo-2020-04-07.tar.bz2’

flecsneurt-demo-202 100%[===================>]  41.98M  13.4MB/s    in 3.7s    

2021-02-04 02:12:24 (11.5 MB/s) - ‘flecsneurt-demo-2020-04-07.tar.bz2’ saved [44021208/44021208]



In [6]:
# Unpacking it
!tar jxvf flecsneurt-demo-2020-04-07.tar.bz2

x collections/squad/derived_data/
x collections/squad/derived_data/embeddings/
x collections/squad/derived_data/bitext/
x collections/squad/derived_data/bitext/question_text
x collections/squad/derived_data/bitext/answer_text_unlemm
x collections/squad/derived_data/bitext/question_text_unlemm
x collections/squad/derived_data/bitext/answer_text
x collections/squad/derived_data/embeddings/starspace/
x collections/squad/derived_data/embeddings/starspace/starspace_unlemm.answer
x collections/squad/derived_data/embeddings/starspace/dim=100_squad_emb_unlemm_epoch1.answer
x collections/squad/derived_data/embeddings/starspace/dim=100_squad_emb_unlemm_epoch1.query
x collections/squad/derived_data/embeddings/starspace/starspace_unlemm.query
x collections/squad/forward_index/
x collections/squad/input_data/
x collections/squad/input_data/train_bitext/
x collections/squad/input_data/dev1/
x collections/squad/input_data/test/
x collections/squad/input_data/dev2/
x collections/squad/input_data/train

In [7]:
# Creating a Lucene index
!scripts/index/create_lucene_index.sh squad

Using collection root: collections
Data directory: collections/squad/input_data
Index directory: collections/squad/lucene_index
Checking data sub-directory: dev1
Found indexable data file: dev1/AnswerFields.jsonl
Checking data sub-directory: dev2
Found indexable data file: dev2/AnswerFields.jsonl
Checking data sub-directory: test
Found indexable data file: test/AnswerFields.jsonl
Checking data sub-directory: train
Found indexable data file: train/AnswerFields.jsonl
Checking data sub-directory: train_bitext
Found indexable data file: train_bitext/AnswerFields.jsonl
Found query file: dev1/QuestionFields.jsonl
Found query file: dev2/QuestionFields.jsonl
Found query file: test/QuestionFields.jsonl
Found query file: train/QuestionFields.jsonl
Found query file: train_bitext/QuestionFields.jsonl
Using the data input file: AnswerFields.jsonl
JAVA_OPTS=-Xms8388608k -Xmx14680064k -server
Creating a new Lucene index, maximum # of docs to process: 2147483647
Input file name: collections/squad/inpu

In [8]:
#!creating a forward index for two fields:
# text is a parsed text field
# text_raw is a raw text field that keeps the text as is
# -clean removes all previous forward indices
!scripts/index/create_fwd_index.sh squad mapdb  \
                               "text:parsedText text_unlemm:raw" \
                               -clean

Using collection root: collections
Data directory:            collections/squad/input_data
Forward index directory:   collections/squad/forward_index/
Clean old index?:          1
Removing previously created index (if exists)
Field list definition:     text:parsedText text_unlemm:raw
Checking data sub-directory: dev1
Found indexable data file: dev1/AnswerFields.jsonl
Checking data sub-directory: dev2
Found indexable data file: dev2/AnswerFields.jsonl
Checking data sub-directory: test
Found indexable data file: test/AnswerFields.jsonl
Checking data sub-directory: train
Found indexable data file: train/AnswerFields.jsonl
Checking data sub-directory: train_bitext
Found indexable data file: train_bitext/AnswerFields.jsonl
Found query file: dev1/QuestionFields.jsonl
Found query file: dev2/QuestionFields.jsonl
Found query file: test/QuestionFields.jsonl
Found query file: train/QuestionFields.jsonl
Found query file: train_bitext/QuestionFields.jsonl
JAVA_OPTS=-Xms12582912k -Xmx14680064k -serv

## API demo

In [9]:
from scripts.py_flexneuart.setup import *

In [10]:
# add Java JAR to the class path
configure_classpath('target')

In [11]:
# create a resource manager
resource_manager=create_featextr_resource_manager('collections/squad/forward_index')

### Retrieval

In [13]:
from scripts.py_flexneuart.cand_provider import *
# create a candidate provider/generator
cand_prov = create_cand_provider(resource_manager, PROVIDER_TYPE_LUCENE, 'collections/squad/lucene_index')

In [14]:
run_query(cand_prov, 20, "university notre dame student run".split())

(1961,
 [CandidateEntry(doc_id='@4309', score=27.225019454956055),
  CandidateEntry(doc_id='@4323', score=27.155115127563477),
  CandidateEntry(doc_id='@2608', score=26.603111267089844),
  CandidateEntry(doc_id='@4310', score=26.364826202392578),
  CandidateEntry(doc_id='@4303', score=26.196977615356445),
  CandidateEntry(doc_id='@4350', score=25.074060440063477),
  CandidateEntry(doc_id='@4346', score=24.675006866455078),
  CandidateEntry(doc_id='@4316', score=24.361064910888672),
  CandidateEntry(doc_id='@4336', score=23.92790985107422),
  CandidateEntry(doc_id='@4345', score=23.00181007385254),
  CandidateEntry(doc_id='@4320', score=22.964710235595703),
  CandidateEntry(doc_id='@2607', score=22.55484390258789),
  CandidateEntry(doc_id='@4325', score=22.466575622558594),
  CandidateEntry(doc_id='@4330', score=21.723785400390625),
  CandidateEntry(doc_id='@4348', score=21.596769332885742),
  CandidateEntry(doc_id='@4321', score=21.47646713256836),
  CandidateEntry(doc_id='@4338', scor

#### Forward index demo

In [16]:
from scripts.py_flexneuart.fwd_index import get_forward_index

### First let's play with a raw index that keeps ony unparsed text

In [17]:
raw_indx = get_forward_index(resource_manager, 'text_unlemm')

In [18]:
# the raw flag is set
raw_indx.is_raw

True

In [19]:
raw_indx.get_doc_raw('@4302')

'architecturally school catholic character atop main building gold dome golden statue virgin mary immediately main building facing copper statue christ arms upraised legend venite ad omnes main building basilica sacred heart immediately basilica grotto marian place prayer reflection replica grotto lourdes france virgin mary reputedly appeared saint bernadette soubirous 1858 end main drive direct line connects 3 statues gold dome simple modern stone statue mary'

### A parsed index has more info

In [20]:
parsed_indx = get_forward_index(resource_manager, 'text')

In [21]:
# here is_raw is False
parsed_indx.is_raw

False

In [22]:
parsed_indx.get_doc_parsed('@4302')

DocEntryParsed(word_ids=[1, 470, 480, 549, 770, 848, 857, 867, 1143, 1193, 1291, 1514, 1562, 1597, 1897, 2210, 2425, 2513, 2579, 3171, 3207, 3357, 3806, 3899, 3960, 4056, 4334, 4790, 5881, 6258, 6274, 6629, 6645, 7051, 7557, 8066, 9139, 10063, 11826, 12878, 13240, 16221, 20752, 32578, 32579, 32580, 32581, 32582, 32583], word_qtys=[1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 4, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 4, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1], word_id_seq=[10063, 1, 6274, 848, 8066, 1291, 1193, 6645, 9139, 3171, 11826, 1143, 4334, 1597, 1291, 1193, 2210, 3899, 11826, 2425, 3806, 32578, 857, 32579, 7051, 13240, 1291, 1193, 4790, 7557, 4056, 1597, 4790, 32580, 6258, 1897, 5881, 16221, 6629, 32580, 32581, 3207, 1143, 4334, 20752, 470, 2579, 32582, 32583, 12878, 1562, 1291, 3357, 867, 770, 2513, 549, 11826, 6645, 9139, 1514, 480, 3960, 11826, 4334], doc_len=65)

In [23]:
# Let's extract the first document word and its info
parsed_indx.get_word_by_id(10063), parsed_indx.get_word_entry_by_id(10063)

('architecturally', WordEntry(word_id=10063, word_freq=11))